In [ ]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
import statsmodels.formula.api as smf

# Build `GGE_FA6_Cleaned_with_s.txt` from `FA06_All.xls`

**Input**: `FA06_All.xls` (FlowJo exports + `Plate_Layout_FA6`)

**Output**: `GGE_FA6_Cleaned_with_s.txt`

In [ ]:
FA06_XLS = Path('FA06_All.xls')
OUT_TXT  = Path('GGE_FA6_Cleaned_with_s.txt')

xls = pd.ExcelFile(FA06_XLS, engine='xlrd')

# --- helpers: reproduce Demultiplex_384W mapping and environment assignment ---
rows384 = list('ABCDEFGHIJKLMNOP')  # 16 rows
rows96  = list('ABCDEFGH')         # 8 rows

def well96_from_384(w384: str) -> str:
    """Map a 384-well coordinate (A1..P24) to a 96-well coordinate (A01..H12).
    This matches what happens after `Demultiplex_384W(..., flat=TRUE)`.
    """
    row = w384[0]
    col = int(w384[1:])
    r_i = rows384.index(row) + 1
    r96 = rows96[(r_i - 1) // 2]
    c96 = (col + 1) // 2
    return f"{r96}{c96:02d}"

def comp_env_from_384(w384: str) -> str:
    """Environment assignment used in ParsingData.R:
    pl1=YPD, pl2=SC_37C, pl3=SCpH73, pl4=SC_NaCl
    which corresponds to (row parity, col parity) in the 384 coordinate.
    """
    row = w384[0]
    col = int(w384[1:])
    r_i = rows384.index(row) + 1
    if (r_i % 2 == 1) and (col % 2 == 1):
        return 'YPD'
    if (r_i % 2 == 1) and (col % 2 == 0):
        return 'SC_37C'
    if (r_i % 2 == 0) and (col % 2 == 1):
        return 'SCpH73'
    return 'SC_NaCl'

def norm384(w384: str) -> str:
    """Match the formatting in the R output: A01, A02, ..."""
    return f"{w384[0]}{int(w384[1:]):02d}"

In [ ]:
# --- parse one FlowJo export sheet (384 wells * 2 rows: total + /Reference) ---
def parse_flowjo_sheet(sheet_name: str) -> pd.DataFrame:
    # Uses columns (1,2,4) => Name + #Cells (we don't need the extra plate column)
    df = pd.read_excel(xls, sheet_name=sheet_name, usecols=[1, 3])
    df.columns = ['Name', 'Cell_counts']

    is_ref = df['Name'].astype(str).str.contains('/Reference', na=False)

    # Extract 384-well ID from the Name field (token after Specimen_###_)
    # e.g. "Specimen_001_A21_A21.fcs" -> "A21"
    def extract_384(name: str) -> str:
        m = re.search(r"Specimen_\d+_([A-P]\d{1,2})_", str(name))
        return m.group(1) if m else np.nan

    df['Well_id_384'] = df['Name'].apply(extract_384)
    df['Well_id_96']  = df['Well_id_384'].apply(lambda w: well96_from_384(w) if isinstance(w, str) else np.nan)

    df_all = df.loc[~is_ref, ['Well_id_384', 'Well_id_96', 'Cell_counts']].rename(columns={'Cell_counts': 'nAll'})
    df_ref = df.loc[ is_ref, ['Well_id_384', 'Cell_counts']].rename(columns={'Cell_counts': 'nRef'})

    out = df_all.merge(df_ref, on='Well_id_384', how='left')

    out['Plate'] = sheet_name[:3]  # "P1_", ..., "P9_", "P10"
    out['Comp_env'] = out['Well_id_384'].apply(comp_env_from_384)
    out['time_point'] = 1 if sheet_name.endswith('_t1') else 2

    # Downstream uses Well_id_384 formatted like "A01"
    out['Well_id_384'] = out['Well_id_384'].apply(norm384)

    return out[['Plate', 'Well_id_96', 'Well_id_384', 'Comp_env', 'time_point', 'nRef', 'nAll']]

flow_sheets = [s for s in xls.sheet_names if s != 'Plate_Layout_FA6']
parsed = pd.concat([parse_flowjo_sheet(s) for s in flow_sheets], ignore_index=True)
#parsed.shape

In [ ]:
# --- reshape to wide (t1/t2 on one row) ---
t1 = parsed[parsed.time_point == 1].drop(columns=['time_point']).rename(columns={'nRef': 'nRef_t1', 'nAll': 'nAll_t1'})
t2 = parsed[parsed.time_point == 2].drop(columns=['time_point']).rename(columns={'nRef': 'nRef_t2', 'nAll': 'nAll_t2'})

wide = t1.merge(t2, on=['Plate', 'Well_id_96', 'Well_id_384', 'Comp_env'], how='inner')

# --- attach layout info (Type, Parent_id, Evolution) ---
layout = pd.read_excel(xls, sheet_name='Plate_Layout_FA6')
layout = layout[['Well_id_96', 'Type', 'Parent_id', 'Evolution']]
wide = wide.merge(layout, on='Well_id_96', how='left')

# FlowJo missing marker used in the R scripts
wide.loc[wide['nRef_t1'] == 9999999, 'nRef_t1'] = np.nan
wide.loc[wide['nAll_t1'] == 9999999, 'nAll_t1'] = np.nan

# Fitness vs reference
dt = 20 - 10
num = ((wide['nAll_t2'] - wide['nRef_t2']) / wide['nRef_t2'])
den = ((wide['nAll_t1'] - wide['nRef_t1']) / wide['nRef_t1'])
wide['s_vs_Ref'] = np.log(num / den) / dt
#wide.shape

In [ ]:
# --- SanityCheckCleaning.R (flags + cleaning) ---
min_events = 2000

min_ref_start = 0.1
max_ref_start = 0.8

min_ref_end = 0.05
max_ref_end = 0.95

G = wide.copy()

# flags
G['low_events']  = ((G['nAll_t1'] < min_events) | (G['nAll_t2'] < min_events)).astype(int)
G['high_ref_t1'] = 0
G['low_ref_t1']  = (G['nRef_t1'] / G['nAll_t1'] < min_ref_start).astype(int)
G['high_ref_t2'] = (G['nRef_t2'] / G['nAll_t2'] > max_ref_end).astype(int)
G['low_ref_t2']  = (G['nRef_t2'] / G['nAll_t2'] < min_ref_end).astype(int)

# High ref at t1:
# Block 1: remove everything except Offspring/Adapted_Parent, and flag those.
ref_frac_t1 = G['nRef_t1'] / G['nAll_t1']
remove_mask1 = (ref_frac_t1 > max_ref_start) & (~G['Type'].isin(['Offspring', 'Adapted_Parent']))
G = G.loc[~remove_mask1].copy()
ref_frac_t1 = G['nRef_t1'] / G['nAll_t1']
G.loc[(ref_frac_t1 > max_ref_start) & (G['Type'].isin(['Offspring', 'Adapted_Parent'])), 'high_ref_t1'] = 1

# Block 2 ("Alternative solution"): recompute ToRemove/ToFlag, but since Block 1 already removed ancestors,
# this effectively just re-flags the remaining rows. We reproduce that behavior.
ref_frac_t1 = G['nRef_t1'] / G['nAll_t1']
remove_mask2 = (ref_frac_t1 > max_ref_start) & (~G['Type'].isin(['Offspring', 'Adapted_Parent', 'Anc_alpha', 'Anc_a']))
G = G.loc[~remove_mask2].copy()
ref_frac_t1 = G['nRef_t1'] / G['nAll_t1']
G['high_ref_t1'] = ((ref_frac_t1 > max_ref_start) & (G['Type'].isin(['Offspring', 'Adapted_Parent', 'Anc_alpha', 'Anc_a']))).astype(int)

# Recompute these flags after any row removals
G['low_ref_t1']  = (G['nRef_t1'] / G['nAll_t1'] < min_ref_start).astype(int)
G['high_ref_t2'] = (G['nRef_t2'] / G['nAll_t2'] > max_ref_end).astype(int)
G['low_ref_t2']  = (G['nRef_t2'] / G['nAll_t2'] < min_ref_end).astype(int)

# Evaporation exclusions
def r_paste_row0(letter, nums):
    return [f"{letter}0{n}" for n in nums]

exclude_evap = {
    1: [*(f"{l}01" for l in "ABCDEFGH"), "A02", *r_paste_row0("H", range(1, 13))],
    2: ["A12", "B12", "C12"],
    3: ["A1", "A2", "A11", "H12", "A12", "B12", "C12"],
    4: ["H12", "A12", "A1"],
    5: [*r_paste_row0("E", range(1, 13)), *r_paste_row0("F", range(1, 13)),
        *r_paste_row0("G", range(1, 13)), *r_paste_row0("H", range(1, 13))],
    6: [*r_paste_row0("A", range(1, 13)), "B01", "B12", "C01", "C12"],
    7: [*(f"{l}01" for l in "ABCDEFGH"), *r_paste_row0("A", [2,3,4,10,11,12])],
    8: ["A01", "A02", "A03", "A12", "B01"],
    9: ["F1", "G1", *r_paste_row0("H", range(1, 13))],
    10: ["A12"],
}

plates = ['P1_', 'P2_', 'P3_', 'P4_', 'P5_', 'P6_', 'P7_', 'P8_', 'P9_', 'P10']

for i, plate in enumerate(plates, start=1):
    wells = set(exclude_evap[i])
    mask = (G['Plate'] == plate) & (G['Comp_env'] == 'SC_37C') & (G['Well_id_96'].isin(wells))
    G = G.loc[~mask].copy()

# Remove too few events
G = G.loc[G['low_events'] != 1].copy()

# Remove wells where ref started low AND ended low
G = G.loc[~((G['low_ref_t2'] == 1) & (G['low_ref_t1'] == 1))].copy()
#G.shape

In [ ]:
# --- ancestor handling + compute s ---

# Remove remaining outlier among Anc_alpha
out_idx = G.index[(G['Type'] == 'Anc_alpha') & (G['s_vs_Ref'] < -0.14)].tolist()
G = G.drop(index=out_idx)

# Replace P2_ Anc_alpha SCpH73 by mean across all other plates
mean_other = G.loc[(G['Plate'] != 'P2_') & (G['Type'] == 'Anc_alpha') & (G['Comp_env'] == 'SCpH73'), 's_vs_Ref'].mean()
G.loc[(G['Plate'] == 'P2_') & (G['Type'] == 'Anc_alpha') & (G['Comp_env'] == 'SCpH73'), 's_vs_Ref'] = mean_other

# Ancestor fitness per plate+env (using only Anc_alpha), then compute s
anc = (G.loc[G['Type'] == 'Anc_alpha']
         .groupby(['Comp_env', 'Plate'], as_index=False)['s_vs_Ref']
         .mean()
         .rename(columns={'s_vs_Ref': 'Anc_Fit'}))

G = G.merge(anc, on=['Plate', 'Comp_env'], how='left')
G['s'] = G['s_vs_Ref'] - G['Anc_Fit']

G['unique_cond'] = G['Plate'] + '_' + G['Comp_env']

# Final column order
out = G[['Parent_id','Well_id_96','Well_id_384','Type','Plate','Evolution','Comp_env',
         'nRef_t1','nAll_t1','nRef_t2','nAll_t2','s_vs_Ref',
         'low_events','high_ref_t1','low_ref_t1','high_ref_t2','low_ref_t2',
         'unique_cond','Anc_Fit','s']].copy()

#out.shape

In [ ]:

# --- write output ---

out_to_write = out.copy()

# --- match row order exactly ---

# Plate order: P1_..P9_, then P10 (no underscore)
plate_order = {f"P{i}_": i for i in range(1, 10)}
plate_order["P10"] = 10

# Comp_env order
env_order = {
    "YPD": 0,
    "SC_37C": 1,
    "SCpH73": 2,
    "SC_NaCl": 3,
}

out_to_write["_plate_num"] = out_to_write["Plate"].map(plate_order)
out_to_write["_env_num"] = out_to_write["Comp_env"].map(env_order)

out_to_write["_well_row"] = out_to_write["Well_id_384"].astype(str).str.extract(r"^([A-H])", expand=False)
out_to_write["_well_col"] = pd.to_numeric(
    out_to_write["Well_id_384"].astype(str).str.extract(r"(\d+)$", expand=False),
    errors="coerce"
)

out_to_write = (
    out_to_write
    .sort_values(
        ["_plate_num", "_env_num", "_well_row", "_well_col"],
        kind="mergesort"
    )
    .drop(columns=["_plate_num", "_env_num", "_well_row", "_well_col"], errors="ignore")
)

# Reset index so row numbers become "1","2","3",...
out_to_write = out_to_write.reset_index(drop=True)

# Define which columns should be treated as numeric (so they print unquoted)
numeric_cols = [
    "nRef_t1","nAll_t1","nRef_t2","nAll_t2",
    "s_vs_Ref","low_events",
    "high_ref_t1","low_ref_t1","high_ref_t2","low_ref_t2",
    "Anc_Fit","s",
]
for c in numeric_cols:
    if c in out_to_write.columns:
        out_to_write[c] = pd.to_numeric(out_to_write[c], errors="coerce")

# - header is ONLY the real column names (quoted)
# - each data row begins with quoted row number "1","2",...
# - strings quoted; numbers unquoted; NA unquoted
# - CRLF line endings
def fmt_cell(val):
    if pd.isna(val):
        return "NA"
    # numbers (ints/floats) -> unquoted
    if isinstance(val, (int, np.integer)):
        return str(int(val))
    if isinstance(val, (float, np.floating)):
        # match R-like precision pretty well
        return format(float(val), ".17g")
    # everything else -> quoted string
    return f"\"{str(val)}\""

with open(OUT_TXT, "w", newline="") as f:
    # header (no leading blank field)
    header = "\t".join([f"\"{c}\"" for c in out_to_write.columns])
    f.write(header + "\r\n")

    # rows with leading quoted row number
    for i, row in out_to_write.iterrows():
        rownum = f"\"{i+1}\""
        fields = [fmt_cell(row[c]) for c in out_to_write.columns]
        f.write(rownum + "\t" + "\t".join(fields) + "\r\n")

#print("Wrote:", OUT_TXT.resolve())

# FA06

**Input**: `GGE_FA6_Cleaned_with_s.txt`, `FA06_parent_kk_strain_map.txt`

**Output**: `fa06_F1_final.csv`, `fa06_Parents_final.csv`


In [ ]:

# ---- inputs ----
IN_TXT  = "GGE_FA6_Cleaned_with_s.txt"
KK_MAP  = "FA06_parent_kk_strain_map.txt"

OUT_F1  = "fa06_F1_final.csv"
OUT_PAR = "fa06_Parents_final.csv"


def read_gge_cleaned(path: str) -> pd.DataFrame:
    """
    Reads the R-style TSV (quoted headers/strings, NA tokens, row-number col).
    Drops the leading row-number column if present.
    """
    df = pd.read_csv(
        path,
        sep="\t",
        quotechar='"',
        na_values=["NA"],
        keep_default_na=True,
        dtype=str,  # read as str first; we'll cast selected columns later
    )
    # Drop possible row-number column (blank header or Unnamed)
    if df.columns[0] == "" or str(df.columns[0]).startswith("Unnamed"):
        df = df.iloc[:, 1:]
    return df


def fit_est_from_freq(f0: pd.Series, f1: pd.Series, t: float = 10.0) -> pd.Series:
    """
    Mirrors fit_est in the R header used here:
      s_hat = (logit(f1) - logit(f0)) / t
    """
    eps = 1e-12
    f0c = f0.clip(eps, 1 - eps)
    f1c = f1.clip(eps, 1 - eps)
    return (np.log(f1c / (1 - f1c)) - np.log(f0c / (1 - f0c))) / t


# ---- 1) Load data ----
FA06 = read_gge_cleaned(IN_TXT)

# Cast numeric columns needed for s_hat / QC / downstream calculations
num_cols = ["nRef_t1", "nAll_t1", "nRef_t2", "nAll_t2",
            "high_ref_t1", "low_ref_t1", "high_ref_t2", "low_ref_t2",
            "low_events"]
for c in num_cols:
    if c in FA06.columns:
        FA06[c] = pd.to_numeric(FA06[c], errors="coerce")

# ---- 2) Merge parent_id -> kk_strain_id map and derive kk fields ----
kkmap = pd.read_csv(KK_MAP, sep="\t", dtype=str)
FA06 = FA06.merge(kkmap, on="Parent_id", how="left", sort=False)

# kk_well_id: 3rd underscore-delimited token of kk_strain_id
FA06["kk_well_id"] = FA06["kk_strain_id"].astype(str).str.split("_").str[2]

# kk_pop_id: remove leading [A-Z][0-9]+_ ; then remove trailing _[0-9] (well replicate)
FA06["kk_pop_id"] = (
    FA06["kk_strain_id"]
    .astype(str)
    .str.replace(r"^[A-Z][0-9]+_", "", regex=True)
    .str.replace(r"_[0-9]$", "", regex=True)
)

# regime from kk_pop_id
FA06["regime"] = np.nan
FA06.loc[FA06["kk_pop_id"].str.contains("a3", na=False), "regime"] = "sexual"
FA06.loc[FA06["kk_pop_id"].str.contains("a1.1", na=False), "regime"] = "asexual"

# ---- 3) env / plate naming ----
FA06["env"] = "YPD"
FA06.loc[FA06["Comp_env"] == "SC_37C", "env"]  = "SC37C"
FA06.loc[FA06["Comp_env"] == "SC_NaCl", "env"] = "SC_0.2M_NaCl"
FA06.loc[FA06["Comp_env"] == "SCpH73", "env"]  = "SC_pH7.3"

# plate = env + '_' + Plate (strip trailing underscore from Plate)
FA06["plate"] = FA06["env"].astype(str) + "_" + FA06["Plate"].astype(str).str.replace(r"_$", "", regex=True)

# ---- 4) s_hat from counts ----
f0 = (FA06["nAll_t1"] - FA06["nRef_t1"]) / FA06["nAll_t1"]
f1 = (FA06["nAll_t2"] - FA06["nRef_t2"]) / FA06["nAll_t2"]
FA06["s_hat"] = fit_est_from_freq(f0=f0, f1=f1, t=10.0)

# flags
FA06["F1"] = FA06["Type"].astype(str).str.contains("Offspring", na=False).astype(int)
FA06["parent"] = FA06["Type"].astype(str).str.contains("Parent", na=False).astype(int)

# ---- 5) Plate effect correction ----
# R does: FA06p <- filter(FA06, parent==1) then remove col07 parents, then glmmTMB with dispformula=~env
# and uses ranef(FA06_pm1)[[1]]$plate (random intercepts) as plate_means.

# Ensure these exist and are correct BEFORE this block:
#   FA06 has columns: s_hat, regime, env, kk_strain_id, plate, parent, Well_id_96

FA06p = FA06[FA06["parent"] == 1].copy()
FA06p = FA06p[~FA06p["Well_id_96"].astype(str).str.contains("07", na=False)].copy()

# Make sure R sees factors similarly
FA06p["regime"] = FA06p["regime"].astype("string")
FA06p["env"] = FA06p["env"].astype("string")
FA06p["kk_strain_id"] = FA06p["kk_strain_id"].astype("string")
FA06p["plate"] = FA06p["plate"].astype("string")

# Install/load rpy2 + glmmTMB (Colab)
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rpy2"])

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# Ensure glmmTMB is installed in the R runtime
ro.r('if(!requireNamespace("glmmTMB", quietly=TRUE)) install.packages("glmmTMB", repos="https://cloud.r-project.org")')
ro.r('library(glmmTMB)')

# Send FA06p to R and fit the exact model
ro.globalenv["FA06p"] = pandas2ri.py2rpy(FA06p)

ro.r(r'''
FA06p <- droplevels(FA06p)

# exact model used in FA06_parsing.R:
m <- glmmTMB(
  s_hat ~ regime * env + (1|kk_strain_id) + (1|plate),
  dispformula = ~ env,
  data = FA06p,
  family = gaussian
)

re_plate <- ranef(m)[[1]]$plate
plate_effect_FA06 <- data.frame(
  plate = row.names(re_plate),
  plate_means = re_plate[,1],
  row.names = NULL
)
''')

plate_effect_df = pandas2ri.rpy2py(ro.globalenv["plate_effect_FA06"])

# Merge plate effects back EXACTLY like R: merge(..., by='plate', sort=F)
# In pandas: preserve FA06 row order by doing left merge on FA06.
FA06c = FA06.merge(plate_effect_df, on="plate", how="left", sort=False)
FA06c["fitness_gain"] = FA06c["s_hat"] - FA06c["plate_means"]



# ---- 6) Parents: QC filter + remove col07 + average ----
qc_ok = (
    (FA06c["high_ref_t1"].fillna(0) == 0) &
    (FA06c["low_ref_t1"].fillna(0) == 0) &
    (FA06c["high_ref_t2"].fillna(0) == 0) &
    (FA06c["low_ref_t2"].fillna(0) == 0)
)
FA06p3 = FA06c[(FA06c["parent"] == 1) & qc_ok].copy()

# remove col 07 parents (Well_id_96 contains '07')
FA06p3 = FA06p3[~FA06p3["Well_id_96"].astype(str).str.contains("07", na=False)].copy()

Ps = (
    FA06p3.groupby(["regime", "kk_strain_id", "env"], dropna=False)
    .agg(mu_s=("fitness_gain", "mean"),
         sd_s=("fitness_gain", "std"),
         n_s=("fitness_gain", "size"))
    .reset_index()
)

# ---- 7) F1s: subset ----
F1s = FA06c[(FA06c["F1"] == 1) & (FA06c["parent"] == 0)].copy()

# ---- 8) Environment means (computed from F1 + parent mu_s) ----
F1_subset = F1s[["regime", "kk_strain_id", "env", "fitness_gain"]].copy()
F1_subset = F1_subset.rename(columns={"fitness_gain": "mu_s"})

Ps_for_means = Ps.drop(columns=["sd_s", "n_s"]).copy()
FA06_m = pd.concat([F1_subset, Ps_for_means], ignore_index=True)

env_means = (
    FA06_m.groupby("env", dropna=False)["mu_s"]
    .mean()
    .reset_index()
    .rename(columns={"mu_s": "env_means"})
)

# Merge back
F1s_2 = F1s.merge(env_means, on="env", how="left")
Ps_2  = Ps.merge(env_means, on="env", how="left")

# Centered fitness
F1s_2["fitness_gain_adj"] = F1s_2["fitness_gain"] - F1s_2["env_means"]
Ps_2["fitness_gain_adj"]  = Ps_2["mu_s"] - Ps_2["env_means"]

# F1 unique id
F1s_2["f1_unique_id"] = (
    F1s_2["kk_strain_id"].astype(str) + "_" +
    F1s_2["Plate"].astype(str) + F1s_2["Well_id_96"].astype(str)
)

# ---- 9) Force exact column order (match fa06_*_final.csv) ----

f1_cols = [
    'env','plate','Parent_id','Well_id_96','Well_id_384','Type','Plate','Evolution','Comp_env',
    'nRef_t1','nAll_t1','nRef_t2','nAll_t2','s_vs_Ref','low_events',
    'high_ref_t1','low_ref_t1','high_ref_t2','low_ref_t2',
    'unique_cond','Anc_Fit','s',
    'kk_strain_id','kk_well_id','kk_pop_id','regime',
    's_hat','F1','parent',
    'plate_means','fitness_gain','env_means','fitness_gain_adj','f1_unique_id'
]

parent_cols = [
    'env','regime','kk_strain_id',
    'mu_s','sd_s','n_s',
    'env_means','fitness_gain_adj'
]

# Keep only columns that exist (safety)
F1s_2 = F1s_2[[c for c in f1_cols if c in F1s_2.columns]]
Ps_2  = Ps_2[[c for c in parent_cols if c in Ps_2.columns]]

# ---- write outputs ----
F1s_2.to_csv(OUT_F1, index=False)
Ps_2.to_csv(OUT_PAR, index=False)

print("Wrote:", OUT_F1, "rows:", len(F1s_2), "cols:", F1s_2.shape[1])
print("Wrote:", OUT_PAR, "rows:", len(Ps_2), "cols:", Ps_2.shape[1])

/tmp/ipython-input-857646435.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sexual' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  FA06.loc[FA06["kk_pop_id"].str.contains("a3", na=False), "regime"] = "sexual"
(as ‘lib’ is unspecified)

































	‘/tmp/RtmpL7kHNn/downloaded_packages’



Wrote: fa06_F1_final.csv rows: 2442 cols: 34
Wrote: fa06_Parents_final.csv rows: 32 cols: 8
